# 🚀 Azure AI 추론이 포함된 DeepSeek-R1 모델  🧠

**DeepSeek-R1**은 강화 학습과 감독 미세 조정을 결합한 최신 추론 모델로, 37억 개의 활성 매개 변수와 128K 컨텍스트 창으로 복잡한 추론 작업에 탁월한 성능을 발휘합니다.

이 노트북에서는 다음을 학습합니다:
1. Azure 서버리스 엔드포인트용 ChatCompletionsClient **초기화**하기
2. 추론 추출을 사용하여 DeepSeek-R1과 **채팅**하기
3. 단계별 추론으로 여행 계획 예제를 **구현**
4. 복잡한 시나리오를 위해 128K 컨텍스트 창을 **활용**

## 왜 DeepSeek-R1인가?
- **고급 추론**: 연쇄적 사고 문제 해결에 특화되어 있습니다.
- **매시브 컨텍스트**: 상세한 분석을 위한 128K 토큰 창
- **효율적인 아키텍처**: 총 671억 개 중 37억 개의 활성 매개변수
- **안전 통합**: 콘텐츠 필터링 기능 내장


## 1. 설정 및 인증

필수 패키지:
- `azure-ai-inference`: 채팅 완료용
- `python-dotenv`: 환경 변수용

.env 파일 요구 사항:
```bash
AZURE_INFERENCE_ENDPOINT=<your-endpoint-url>
AZURE_INFERENCE_KEY=<your-api-key>
MODEL_NAME=DeepSeek-R1
```

In [ ]:
import os
import re
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# Load environment
load_dotenv()
endpoint = os.getenv("AZURE_INFERENCE_ENDPOINT")
key = os.getenv("AZURE_INFERENCE_KEY")
model_name = os.getenv("MODEL_NAME", "DeepSeek-R1")

# Initialize client
try:
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key)
    )
    print("✅ Client initialized | Model:", client.get_model_info().model_name)
except Exception as e:
    print("❌ Initialization failed:", e)

## 2. 똑똑한 여행 계획 ✈️

여행 계획을 위한 DeepSeek-R1의 추론 기능을 시연합니다:

In [ ]:
def plan_trip_with_reasoning(query, show_thinking=False):
    """Get travel recommendations with reasoning extraction"""
    messages = [
        SystemMessage(content="You are a travel expert. Provide detailed plans with rationale."),
        UserMessage(content=f"{query} Include hidden gems and safety considerations.")
    ]
    
    response = client.complete(
        messages=messages,
        model=model_name,
        temperature=0.7,
        max_tokens=1024
    )
    
    content = response.choices[0].message.content
    
    # Extract reasoning if present
    if show_thinking:
        match = re.search(r"<think>(.*?)</think>(.*)", content, re.DOTALL)
        if match:
            return {"thinking": match.group(1).strip(), "answer": match.group(2).strip()}
    return content

# Example usage
query = "Plan a 5-day cultural trip to Kyoto in April"
result = plan_trip_with_reasoning(query, show_thinking=True)

print("🗺️ Query:", query)
if isinstance(result, dict):
    print("\n🧠 Thinking Process:", result["thinking"])
    print("\n📝 Final Answer:", result["answer"])
else:
    print("\n📝 Response:", result)

## 3. 기술적 문제 해결 💻

코딩/최적화 기능을 보여주세요:

In [ ]:
def solve_technical_problem(problem):
    """Solve complex technical problems with structured reasoning"""
    response = client.complete(
        messages=[
            UserMessage(content=f"{problem} Please reason step by step, and put your final answer within \boxed{{}}.")
        ],
        model=model_name,
        temperature=0.3,
        max_tokens=2048
    )
    
    return response.choices[0].message.content

# Database optimization example
problem = """How can I optimize a PostgreSQL database handling 10k transactions/second?
Consider indexing strategies, hardware requirements, and query optimization."""

print("🔧 Problem:", problem)
print("\n⚙️ Solution:", solve_technical_problem(problem))

## 4. 모범 사례 및 고려 사항

1. **추론 처리**: 정규식을 사용하여 <think> 콘텐츠와 최종 답변을 구분합니다.
2. **안전**: 빌트인 콘텐츠 필터링 - 위반 시 HttpResponseError 처리
3. **성능**:
   - 최대 토큰: 4096
   - 속도 제한: 분당 200K 토큰
4. **비용**: 서버리스 배포를 통한 종량제 요금제
5. **스트리밍**: 장시간 completion을 위한 응답 스트리밍 구현

```python
# Streaming example
response = client.complete(..., stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")
```

## 🎯 주요 요점
- 자세한 분석을 위해 128K 컨텍스트 활용
- 디버깅/분석을 위한 추론 단계 추출
- 프로덕션을 위해 Azure AI Content Safety와 결합
- response.usage를 통해 토큰 사용량 모니터링

> 중요한 애플리케이션에 대해 항상 모델 출력의 유효성을 검사하세요!